# Extra Credit Solution — Interactive Chatbot with Moderation

This notebook provides solutions for the three extra credit exercises from **Lab Exercise 2**:

1. **Part 1** — Build an interactive chatbot using Groq
2. **Part 2** — Add conversation history so the chatbot remembers context
3. **Part 3** — Add content moderation using meta-prompting

In [ ]:
%pip install groq python-dotenv

In [ ]:
import os
from dotenv import load_dotenv
from groq import Groq

# Load environment variables
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")

# Configure Groq client
client = Groq(api_key=api_key)
model = "llama-3.3-70b-versatile"

## Part 1: Interactive Chatbot

A simple loop that prompts the user for input, sends it to the Groq API, and prints the response. Type `quit` or `exit` to stop.

In [ ]:
print("Chatbot ready! Type 'quit' or 'exit' to stop.\n")

while True:
    user_input = input("You: ")
    if user_input.strip().lower() in ["quit", "exit"]:
        print("Goodbye!")
        break

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": user_input},
        ],
        temperature=0.7,
        max_tokens=256,
    )

    reply = response.choices[0].message.content
    print(f"Assistant: {reply}\n")

## Part 2: Add Conversation History

The chatbot above has no memory — each turn is independent. To give the model context of the full conversation, we maintain a `messages` list and append both the **user** message and the **assistant** reply after every exchange.

The `"assistant"` role represents the model's own previous replies. By including these in the messages list, the model can "remember" what it said earlier and maintain coherent, multi-turn conversations.

**Test it:** Tell the chatbot your name, then in a later turn ask *"What is my name?"* — it should remember.

In [ ]:
print("Chatbot with memory ready! Type 'quit' or 'exit' to stop.\n")

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
]

while True:
    user_input = input("You: ")
    if user_input.strip().lower() in ["quit", "exit"]:
        print("Goodbye!")
        break

    # Append the user's message to the conversation history
    messages.append({"role": "user", "content": user_input})

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7,
        max_tokens=256,
    )

    reply = response.choices[0].message.content

    # Append the assistant's reply to the conversation history
    messages.append({"role": "assistant", "content": reply})

    print(f"Assistant: {reply}\n")

## Part 3: Add Content Moderation

We use **meta-prompting** to make the model judge whether a piece of text contains harmful content (e.g., racism, hate speech, illegal requests, or other inappropriate material).

The `moderate` function sends the text to the model with a system prompt that instructs it to respond with only `"yes"` (harmful) or `"no"` (safe). We parse the response into a boolean.

In [ ]:
def moderate(text):
    """Return True if the text is flagged as harmful, False otherwise."""
    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a content moderation assistant. "
                    "Determine whether the following text contains harmful content "
                    "such as racism, hate speech, sexually explicit material, "
                    "encouragement of violence, illegal activity requests, "
                    "or other inappropriate material. "
                    "Respond with ONLY 'yes' or 'no'. "
                    "'yes' means the content IS harmful. 'no' means it is safe."
                ),
            },
            {"role": "user", "content": text},
        ],
        temperature=0,
        max_tokens=4,
    )
    result = response.choices[0].message.content.strip().lower()
    return result.startswith("yes")

In [ ]:
SAFE_FALLBACK = "I'm sorry, I can't respond to that. Let's keep our conversation respectful and safe."

print("Chatbot with memory + moderation ready! Type 'quit' or 'exit' to stop.\n")

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
]

while True:
    user_input = input("You: ")
    if user_input.strip().lower() in ["quit", "exit"]:
        print("Goodbye!")
        break

    # Moderate the user's input
    if moderate(user_input):
        print(f"Assistant: {SAFE_FALLBACK}\n")
        continue

    # Append user message and get response
    messages.append({"role": "user", "content": user_input})

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7,
        max_tokens=256,
    )

    reply = response.choices[0].message.content

    # Moderate the assistant's response
    if moderate(reply):
        reply = SAFE_FALLBACK

    # Append assistant reply to history
    messages.append({"role": "assistant", "content": reply})

    print(f"Assistant: {reply}\n")